In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num=10):
        super(AlexNet, self).__init__()
        self.feature = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,num),

        )

    def forward(self, x):

        x = self.feature(x)
        x = x.view(-1,32*12*12)
        x = self.classifier(x)
        return x


In [ ]:
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

# Load MNIST dataset
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [ ]:
# Split the training dataset into labeled and unlabeled subsets
labeled_indices = torch.randperm(len(train_dataset))[:600]
unlabeled_indices = torch.randperm(len(train_dataset))[600:10000] #RAM issues
labeled_dataset = Subset(train_dataset, labeled_indices)
unlabeled_dataset = Subset(train_dataset, unlabeled_indices)

# Create data loaders
labeled_loader = DataLoader(labeled_dataset, batch_size=32, shuffle=True)
unlabeled_loader = DataLoader(unlabeled_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Define the model, loss function, and optimizer
model = AlexNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.SGD(model.parameters(), lr=1.5, momentum=0.9)

In [ ]:
# Semi-supervised training function
def train_semi_supervised(model, labeled_loader, unlabeled_loader, test_loader, epochs, alpha_schedule):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    print(device)

    for epoch in range(epochs):
        model.train()
        labeled_loss = 0.0
        unlabeled_loss = 0.0

        # Train on labeled data
        for images, labels in labeled_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            labeled_loss += loss.item()
        labeled_loss /= len(labeled_loader)

        # # Train on unlabeled data with Pseudo-Labels
        # alpha = alpha_schedule(epoch)
        # for images, _ in unlabeled_loader:
        #     images = images.to(device)
        #     optimizer.zero_grad()
        #     outputs = model(images)
        #     pseudo_labels = torch.max(outputs, 1)[1]
        #     unlabeled_loss += alpha * criterion(outputs, pseudo_labels)
        # unlabeled_loss /= len(unlabeled_loader)
        # # Backpropagate the combined loss
        # (labeled_loss + unlabeled_loss).backward()
        # optimizer.step()

        # Evaluate on test set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        test_accuracy = 100 * correct / total
        print(f'Epoch [{epoch+1}/{epochs}], Labeled Loss: {labeled_loss:.4f}, Unlabeled Loss: {unlabeled_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


In [ ]:
# Define the alpha schedule for semi-supervised learning
T1 = 10
T2 = 30
def alpha_schedule(epoch):
    if epoch < T1:
        return 0
    elif epoch < T2:
        return ((epoch - T1) / (T2 - T1)) * 3
    else:
        return 3

In [ ]:
# Train the model using semi-supervised learning
train_semi_supervised(model, labeled_loader, unlabeled_loader, test_loader, epochs=50, alpha_schedule=alpha_schedule)

cuda
Epoch [1/100], Labeled Loss: 2.3236, Unlabeled Loss: 0.0000, Test Accuracy: 10.28%
Epoch [2/100], Labeled Loss: 2.0536, Unlabeled Loss: 0.0000, Test Accuracy: 64.35%
Epoch [3/100], Labeled Loss: 0.9603, Unlabeled Loss: 0.0000, Test Accuracy: 76.29%
Epoch [4/100], Labeled Loss: 0.7035, Unlabeled Loss: 0.0000, Test Accuracy: 88.32%
Epoch [5/100], Labeled Loss: 0.3994, Unlabeled Loss: 0.0000, Test Accuracy: 90.13%
Epoch [6/100], Labeled Loss: 0.2485, Unlabeled Loss: 0.0000, Test Accuracy: 91.11%
Epoch [7/100], Labeled Loss: 0.2287, Unlabeled Loss: 0.0000, Test Accuracy: 91.56%
Epoch [8/100], Labeled Loss: 0.1845, Unlabeled Loss: 0.0000, Test Accuracy: 93.06%
Epoch [9/100], Labeled Loss: 0.1421, Unlabeled Loss: 0.0000, Test Accuracy: 93.66%
Epoch [10/100], Labeled Loss: 0.1138, Unlabeled Loss: 0.0000, Test Accuracy: 94.56%
Epoch [11/100], Labeled Loss: 0.1063, Unlabeled Loss: 0.0000, Test Accuracy: 94.39%
Epoch [12/100], Labeled Loss: 0.0887, Unlabeled Loss: 0.0000, Test Accuracy: 94.